# A solvis demo with the NSHM Composite Solution

In [1]:
import os
import pathlib
import json
from IPython.display import GeoJSON

import nzshm_model as nm
import solvis

warning openquake module dependency not available, maybe you want to install
                with nzshm-model[openquake]


Running without `toshi` options


In [2]:
# prepare our NSHM composite solution
current_model = nm.get_model_version("NSHM_v1.0.4")
slt = current_model.source_logic_tree

In [3]:
# load the composite_solution and shoose the Cruslal FaultSystemSolution
csol = solvis.CompositeSolution.from_archive("NSHM_v1.0.4_CompositeSolution.zip", slt)
fss = csol._solutions['CRU']

In [4]:
# try our new filtering
from solvis.fault_system_solution_helper import FaultSystemSolutionHelper

In [5]:
helper = FaultSystemSolutionHelper(fss)

In [6]:
# help(helper.subsections_for_faults)
subsections = helper.ruptures_for_parent_fault_names(['Aka Aka'])
subsections

{9, 265110}

## display the surface geometry of a fault

In [29]:
surfaces = fss.fault_surfaces()
mro_surface = surfaces[surfaces.ParentName=="Masterton"]

# A fault called `Masterton` having 4 subsections
mro_surface # display the dataframe

,FaultID,FaultName,DipDeg,Rake,LowDepth,UpDepth,DipDir,AseismicSlipFactor,CouplingCoeff,ParentID,ParentName,geometry,Target Slip Rate,Target Slip Rate StdDev
1125,1125,"Masterton, Subsection 0",65.0,-160.0,12.88,0.0,164.0,0.0,1.0,257,Masterton,"POLYGON ((175.73310 -40.92830, 175.72130 -40.9...",0.000652,0.000577
1126,1126,"Masterton, Subsection 1",65.0,-160.0,12.88,0.0,164.0,0.0,1.0,257,Masterton,"POLYGON ((175.66992 -40.93846, 175.60882 -40.9...",0.000788,0.000592
1127,1127,"Masterton, Subsection 2",65.0,-160.0,12.88,0.0,164.0,0.0,1.0,257,Masterton,"POLYGON ((175.60882 -40.96057, 175.58770 -40.9...",0.000652,0.000530
1128,1128,"Masterton, Subsection 3",65.0,-160.0,12.88,0.0,164.0,0.0,1.0,257,Masterton,"POLYGON ((175.54348 -40.97084, 175.54080 -40.9...",0.000518,0.000468


In [8]:
# plot the masterton surfaces
GeoJSON(json.loads(mro_surface.to_json()))

<IPython.display.GeoJSON object>

In [9]:
### get ruptures passing through a fault

In [10]:
mro_ruptures = helper.ruptures_for_parent_fault_names(['Masterton'])

### get rupture fault sections (rs) with rates for those ruptures

In [11]:
df0 = fss.rs_with_rupture_rates
mro_df = df0[df0["Rupture Index"].isin(mro_ruptures)]
mro_df

key_0 fault_system  Rupture Index  rate_max  \
fault_system Rupture Index                                                 
CRU          209026         209026          CRU         209026  0.000023   
             209026         209026          CRU         209026  0.000023   
             209026         209026          CRU         209026  0.000023   
             209026         209026          CRU         209026  0.000023   
             209026         209026          CRU         209026  0.000023   
...                            ...          ...            ...       ...   
             391758         391758          CRU         391758  0.000100   
             391758         391758          CRU         391758  0.000100   
             391758         391758          CRU         391758  0.000100   
             391758         391758          CRU         391758  0.000100   
             391758         391758          CRU         391758  0.000100   

                                rate_min  rate_count  rate_weighted_mean  \
fault_system Rupture Index                                                 
CRU          209026         3.083809e-07        12.0            0.000003   
             209026         3.083809e-07        12.0            0.000003   
             209026         3.083809e-07        12.0            0.000003   
             209026         3.083809e-07        12.0            0.000003   
             209026         3.083809e-07        12.0            0.000003   
...                                  ...         ...                 ...   
             391758         2.606850e-05        18.0            0.000032   
             391758         2.606850e-05        18.0            0.000032   
             391758         2.606850e-05        18.0            0.000032   
             391758         2.606850e-05        18.0            0.000032   
             391758         2.606850e-05        18.0            0.000032   

                            Magnitude  Average Rake (degrees)    Area (m^2)  \
fault_system Rupture Index                                                    
CRU          209026          7.998682              156.755585  6.290518e+09   
             209026          7.998682              156.755585  6.290518e+09   
             209026          7.998682              156.755585  6.290518e+09   
             209026          7.998682              156.755585  6.290518e+09   
             209026          7.998682              156.755585  6.290518e+09   
...                               ...                     ...           ...   
             391758          7.152529              167.700623  8.964618e+08   
             391758          7.152529              167.700623  8.964618e+08   
             391758          7.152529              167.700623  8.964618e+08   
             391758          7.152529              167.700623  8.964618e+08   
             391758          7.152529              167.700623  8.964618e+08   

                               Length (m)  section  
fault_system Rupture Index                          
CRU          209026         292585.343750    725.0  
             209026         292585.343750    724.0  
             209026         292585.343750    723.0  
             209026         292585.343750    722.0  
             209026         292585.343750    721.0  
...                                   ...      ...  
             391758          53762.265625   2106.0  
             391758          53762.265625   2107.0  
             391758          53762.265625   1127.0  
             391758          53762.265625   1207.0  
             391758          53762.265625   1206.0  

[275 rows x 12 columns]

In [12]:
print(f'the `Masterton` fault has {len(mro_df["Rupture Index"].unique())} unique ruptures...')
print()
mro_ids = list(mro_df["Rupture Index"].unique())
print(mro_ids)


the `Masterton` fault has 19 unique ruptures...

[209026, 219862, 273732, 273733, 273738, 287882, 287883, 287884, 287886, 287925, 287931, 287935, 287979, 288084, 288088, 288181, 391515, 391580, 391758]


In [13]:
### display a single rupture

In [14]:
# let's pick one rupture and display it
surface = fss.rupture_surface(mro_ids[1])
GeoJSON(json.loads(surface.to_json()))

<IPython.display.GeoJSON object>